# ФБ-33 Терещенко Микола 
# Лабораторна робота №2

In [1]:
import urllib.request
import pandas as pd
from datetime import datetime
import os
import time
import re
import tabulate

In [2]:
def download_csv(country, year_1, year_2, type_data, directory):
    os.makedirs(directory, exist_ok=True)
    
    for province_ID in range(1, 28):
        url = (f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?"
               f"country={country}&provinceID={province_ID}&year1={year_1}&year2={year_2}&type={type_data}")
        
        retries, delay = 3, 5
        for attempt in range(retries):
            try:
                with urllib.request.urlopen(url) as response:
                    data = response.read()
                break
            except urllib.error.URLError as e:
                print(f"[!] Error downloading ID {province_ID}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                else:
                    print(f"[-] Skipping ID {province_ID} after {retries} failed attempts.")
                    data = None
        
        if data:
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"NOAA_ID_{province_ID}_{timestamp}.csv"
            filepath = os.path.join(directory, filename)
            
            try:
                with open(filepath, 'wb') as file:
                    file.write(data)
                print(f"[+] Downloaded: {filename}")
            except IOError as e:
                print(f"[!] File write error ({filename}): {e}")
    

def clean_directory(directory):
    os.makedirs(directory, exist_ok=True)
    
    if input("Do you want to clean the directory? (y/n): ").strip().lower() == 'y':
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            try:
                os.remove(filepath)
                print(f"[+] Removed: {filename}")
            except Exception as e:
                print(f"[!] Error removing {filename}: {e}")
        print("[+] Directory cleaned successfully.")
    else:
        print("[+] Directory cleaning skipped.")

In [3]:
country="UKR"
year_1=1981
year_2=2024
type_data="Mean"
directory="/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"

clean_directory(directory)    
download_csv(country, year_1, year_2, type_data, directory)

Do you want to clean the directory? (y/n): y
[+] Removed: NOAA_ID_6_2025-03-13_05-21-12.csv
[+] Removed: NOAA_ID_17_2025-03-13_05-21-39.csv
[+] Removed: NOAA_ID_19_2025-03-13_05-21-45.csv
[+] Removed: NOAA_ID_11_2025-03-13_05-21-24.csv
[+] Removed: NOAA_ID_2_2025-03-13_05-21-02.csv
[+] Removed: NOAA_ID_10_2025-03-13_05-21-21.csv
[+] Removed: NOAA_ID_12_2025-03-13_05-21-28.csv
[+] Removed: NOAA_ID_21_2025-03-13_05-21-49.csv
[+] Removed: NOAA_ID_24_2025-03-13_05-21-55.csv
[+] Removed: NOAA_ID_25_2025-03-13_05-21-57.csv
[+] Removed: NOAA_ID_16_2025-03-13_05-21-37.csv
[+] Removed: NOAA_ID_9_2025-03-13_05-21-19.csv
[+] Removed: NOAA_ID_13_2025-03-13_05-21-30.csv
[+] Removed: NOAA_ID_3_2025-03-13_05-21-04.csv
[+] Removed: NOAA_ID_23_2025-03-13_05-21-53.csv
[+] Removed: NOAA_ID_7_2025-03-13_05-21-14.csv
[+] Removed: NOAA_ID_14_2025-03-13_05-21-32.csv
[+] Removed: NOAA_ID_1_2025-03-13_05-21-00.csv
[+] Removed: NOAA_ID_4_2025-03-13_05-21-06.csv
[+] Removed: NOAA_ID_5_2025-03-13_05-21-09.csv
[+]

In [4]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)

def read_csv_file(filepath):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    try:
        df = pd.read_csv(filepath, header=1, names=headers, converters={'Year': remove_html_tags})
        df = df[df['VHI'] != -1].drop(columns=['empty']).iloc[:-1]
        return df
    except pd.errors.ParserError as e:
        print(f"[!] Error reading {filepath}: {e}")
        return None

def read_data(directory):
    data_frames = []
    for filename in filter(lambda f: f.endswith('.csv'), os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        parts = filename.split('_')
        province_ID = int(parts[2]) if len(parts) >= 3 and parts[2].isdigit() else None
        
        if province_ID and province_ID not in [12, 20]:
            df = read_csv_file(filepath)
            if df is not None:
                df.insert(0, "PROVINCE_ID", province_ID)
                df = df.astype({"Week": int, "Year": int})
                data_frames.append(df)
    
    data_frames.sort(key=lambda x: x["PROVINCE_ID"].iloc[0])
    return pd.concat(data_frames).drop_duplicates().reset_index(drop=True) if data_frames else pd.DataFrame()

def print_dataframe(df):
    print(tabulate.tabulate(df, headers='keys', tablefmt='heavy_grid', showindex=False))

In [5]:
directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
data_frames_1 = read_data(directory)
print_dataframe(data_frames_1.head(10))
print_dataframe(data_frames_1.tail(10))  

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 ┃ 26.65 ┃ 34.06 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫

In [6]:
def searching(directory):
    for file_name in filter(lambda f: f.endswith('.csv'), os.listdir(directory)):
        search_id_city(os.path.join(directory, file_name))

def search_id_city(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        search_results = re.findall(r"= (\d+): ([\w'\s-]+),", content)
        print(search_results)
        return search_results
    except IOError as e:
        print(f"[!] Error reading {file_path}: {e}")
        return []

In [7]:
directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
searching(directory)

[('11', 'Kiev')]
[('9', 'Kherson')]
[('6', "Donets'k")]
[('15', "L'viv")]
[('17', 'Odessa')]
[('23', 'Transcarpathia')]
[('14', "Luhans'k")]
[('13', 'Kirovohrad')]
[('18', 'Poltava')]
[('12', 'Kiev City')]
[('21', 'Sumy')]
[('25', 'Volyn')]
[('1', 'Cherkasy')]
[('20', "Sevastopol'")]
[('5', "Dnipropetrovs'k")]
[('16', 'Mykolayiv')]
[('27', 'Zhytomyr')]
[('19', 'Rivne')]
[('7', "Ivano-Frankivs'k")]
[('4', 'Crimea')]
[('8', 'Kharkiv')]
[('22', "Ternopil'")]
[('10', "Khmel'nyts'kyy")]
[('24', 'Vinnytsya')]
[('2', 'Chernihiv')]
[('26', 'Zaporizhzhya')]
[('3', 'Chernivtsi')]


In [8]:
def replace_function(data_frames):
    province_ID_dictionary = {
        16: 13, 27: 5, 10: 21, 3: 23, 11: 9, 25: 2, 17: 14, 21: 17, 1: 22,    
        18: 15, 6: 4, 9: 20, 8: 19, 19: 16, 23: 6, 7: 8, 5: 3, 26: 7, 2: 24,
        4: 25, 15: 11, 22: 18, 13: 10, 24: 1, 14: 12
    }    
    return data_frames.copy().replace({"PROVINCE_ID": province_ID_dictionary})

In [9]:
directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
data_frames = read_data(directory)        
data_frames_replace = replace_function(data_frames) 
print_dataframe(data_frames_replace.head(10))
print_dataframe(data_frames_replace.tail(10))

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 ┃ 26.65 ┃ 34.06 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫

In [10]:
def VHI_extremes(province_ID, year_1, year_2, df):
    PROVINCE_NAME_dict = {
        1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
        6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
        11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
        16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
        21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
    }
    
    PROVINCE_NAME = PROVINCE_NAME_dict.get(province_ID, 'Невідома область')
    filtered_df = df[(df["PROVINCE_ID"] == province_ID) & (df["Year"].between(year_1, year_2))]
    vhi_values_filtered = filtered_df['VHI'].replace(-1, pd.NA).dropna()
    
    min_value = vhi_values_filtered.min() if not vhi_values_filtered.empty else None
    max_value = vhi_values_filtered.max() if not vhi_values_filtered.empty else None
    
    return [province_ID, PROVINCE_NAME, min_value, max_value]

def VHI_main():
    directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
    data_frames = replace_function(read_data(directory))
    
    year_1, year_2 = 2024, 2024
    headers = ["Province ID", "PROVINCE_NAME", "Min VHI", "Max VHI"]
    vhi_extremes_data = [VHI_extremes(province_ID, year_1, year_2, data_frames) for province_ID in range(1, 26)]
    
    print(tabulate.tabulate(vhi_extremes_data, headers=headers, tablefmt="github"))

In [11]:
VHI_main()

|   Province ID | PROVINCE_NAME     |   Min VHI |   Max VHI |
|---------------|-------------------|-----------|-----------|
|             1 | Вінницька         |     28.83 |     54.56 |
|             2 | Волинська         |     39.94 |     62.34 |
|             3 | Дніпропетровська  |     19    |     52.22 |
|             4 | Донецька          |     24.36 |     50.33 |
|             5 | Житомирська       |     37.66 |     56.83 |
|             6 | Закарпатська      |     37.49 |     61.48 |
|             7 | Запорізька        |     20.31 |     47.59 |
|             8 | Івано-Франківська |     40.11 |     56.71 |
|             9 | Київська          |     33.41 |     55.4  |
|            10 | Кіровоградська    |     21.08 |     49.51 |
|            11 | Луганська         |     39.9  |     58.22 |
|            12 | Львівська         |     24.69 |     50.97 |
|            13 | Миколаївська      |     22.31 |     55.45 |
|            14 | Одеська           |     28.42 |     68.43 |
|       

In [12]:
def VHI_extremes(province_ID, year, df):
    PROVINCE_NAME_dict = {
        1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
        6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
        11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
        16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
        21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
    }
    
    PROVINCE_NAME = PROVINCE_NAME_dict.get(province_ID, "Unknown")
    filtered_df = df.query("PROVINCE_ID == @province_ID and Year == @year")
    vhi_values_filtered = filtered_df['VHI'].replace(-1, pd.NA).dropna()
    
    min_value, max_value = (vhi_values_filtered.min(), vhi_values_filtered.max()) if not vhi_values_filtered.empty else (None, None)
    
    return [province_ID, PROVINCE_NAME, year, min_value, max_value, vhi_values_filtered.tolist()[:10]]

def Task_main():
    directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
    data_frames_replace = replace_function(read_data(directory))
    
    year = 2020
    headers = ["Province ID", "PROVINCE_NAME", "Year", "Min VHI", "Max VHI", "VHI"]
    vhi_extremes_data = [VHI_extremes(province_ID, year, data_frames_replace) for province_ID in range(1, 26)]
    
    print(tabulate.tabulate(vhi_extremes_data, headers=headers, tablefmt="github"))

In [13]:
Task_main()

|   Province ID | PROVINCE_NAME     |   Year |   Min VHI |   Max VHI | VHI                                                                    |
|---------------|-------------------|--------|-----------|-----------|------------------------------------------------------------------------|
|             1 | Вінницька         |   2020 |     34.48 |     64.12 | [40.92, 43.19, 44.74, 45.29, 44.8, 43.92, 43.1, 42.88, 43.71, 45.61]   |
|             2 | Волинська         |   2020 |     39.48 |     61.93 | [48.95, 49.23, 49.29, 49.4, 49.75, 50.26, 51.12, 51.59, 51.52, 51.72]  |
|             3 | Дніпропетровська  |   2020 |     19.4  |     68.14 | [47.23, 46.99, 46.92, 46.55, 45.87, 45.14, 43.96, 43.39, 43.41, 43.13] |
|             4 | Донецька          |   2020 |     17.88 |     59.14 | [44.72, 46.28, 47.98, 48.67, 48.03, 46.47, 43.85, 42.14, 40.76, 39.92] |
|             5 | Житомирська       |   2020 |     35.05 |     58.33 | [42.82, 43.69, 44.52, 45.71, 45.62, 45.13, 45.76, 46.88, 47.05, 4

In [14]:
def VHI_extremes(province_ID, year_1, year_2, df):
    
    PROVINCE_NAME_dict = {1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
        6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
        11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
        16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
        21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'}
    
    PROVINCE_NAME = PROVINCE_NAME_dict[province_ID]
    
    filtered_df = df[(df["PROVINCE_ID"] == province_ID) & (df["Year"] >= year_1) & (df["Year"] <= year_2)]
    vhi_values = filtered_df['VHI']  
    
    return [province_ID, PROVINCE_NAME, year_1, year_2, vhi_values.tolist()[:10]]

def check_extreme_and_moderate(df, threshold_percentage, VHI):
    extreme_years = []
    years = sorted(df['Year'].unique())
    total_provinces = len(df['PROVINCE_ID'].unique())

    for year in years:
        condition = (df['Year'] == year) & (df['VHI'] <= VHI[1]) & (df['VHI'] >= VHI[0]) & (df['VHI'] != -1) 
        drought_count = df[condition].groupby('PROVINCE_ID').size().count()
        if (drought_count / total_provinces) * 100 > threshold_percentage:
            extreme_years.append(year)

    return extreme_years


def Task_main_2():    
    directory = "/home/kali/Desktop/data_analysis/workspace/lab_2_venv/data_csv"
    data_frames_1 = read_data(directory)
    data_frames_replace = replace_function(data_frames_1)

    year_1 = 1982
    year_2 = 2024  
        
    vhi_extremes_data = []
    headers = ["Province ID", "PROVINCE_NAME", "Year_Start", "Year_End", "VHI"]
    
    for province_ID in range(1, 11):  
        vhi_extremes_data.append(VHI_extremes(province_ID, year_1, year_2, data_frames_replace))    
    print(tabulate.tabulate(vhi_extremes_data, headers=headers, tablefmt="github"))

    VHI=[0, 15]
    extreme_years = check_extreme_and_moderate(data_frames_replace, 19, VHI)  # Extreme droughts
    print(f"="*155)
    print(f"Extreme drought years affecting more than 20% of provinces:", extreme_years)
    print(f"="*155)
        
    VHI=[15, 35]    
    moderate_years = check_extreme_and_moderate(data_frames_replace, 20, VHI)  # Moderate droughts
    print(f"Moderate drought years affecting more than 20% of provinces:", moderate_years)
    print(f"="*155)

In [15]:
Task_main_2()

|   Province ID | PROVINCE_NAME     |   Year_Start |   Year_End | VHI                                                                    |
|---------------|-------------------|--------------|------------|------------------------------------------------------------------------|
|             1 | Вінницька         |         1982 |       2024 | [45.9, 45.34, 44.88, 41.6, 39.29, 37.65, 35.03, 34.46, 35.04, 34.33]   |
|             2 | Волинська         |         1982 |       2024 | [53.88, 50.97, 45.65, 40.04, 36.44, 35.14, 33.57, 33.23, 33.53, 35.07] |
|             3 | Дніпропетровська  |         1982 |       2024 | [51.36, 47.14, 42.35, 38.53, 36.19, 34.71, 33.35, 33.65, 34.17, 34.27] |
|             4 | Донецька          |         1982 |       2024 | [55.79, 51.93, 47.95, 43.2, 40.98, 40.76, 39.84, 37.98, 36.65, 35.87]  |
|             5 | Житомирська       |         1982 |       2024 | [49.5, 48.62, 48.86, 46.28, 43.61, 41.71, 38.74, 37.61, 36.98, 36.44]  |
|             6 | Закарпатс